# Get POIs from OSM Data via Overpass API

Import libraries

In [1]:
# Import necessary libraries
import overpy
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from pathlib import Path
import os
import pyogrio

Access Retirement Homes in Heidelberg via overpass API

In [2]:
# Build query to retrieve data for retirement homes in Heidelberg from OpenStreetMap
api = overpy.Overpass()
result = api.query("""
    [out:json];
    area["name"="Heidelberg"]["admin_level"="6"]->.searchArea;
    (
      node["amenity"="retirement_home"](area.searchArea);
      way["amenity"="retirement_home"](area.searchArea);
      relation["amenity"="retirement_home"](area.searchArea);
      node["amenity"="nursing_home"](area.searchArea);
      way["amenity"="nursing_home"](area.searchArea);
      relation["amenity"="nursing_home"](area.searchArea);
      node["amenity"="social_facility"]["social_facility"="nursing_home"](area.searchArea);
      way["amenity"="social_facility"]["social_facility"="nursing_home"](area.searchArea);
      relation["amenity"="social_facility"]["social_facility"="nursing_home"](area.searchArea);
      node["amenity"="social_facility"]["social_facility"="assisted_living"](area.searchArea);
      way["amenity"="social_facility"]["social_facility"="assisted_living"](area.searchArea);
      relation["amenity"="social_facility"]["social_facility"="assisted_living"](area.searchArea);
    );
    out center;
""")

# Prepare data for GeoDataFrame
data = []

for element in result.nodes + result.ways + result.relations:
    name = element.tags.get('name', 'N/A')
    amenity_type = element.tags.get('amenity', 'N/A')
    social_facility_type = element.tags.get('social_facility', 'N/A')
    
    if isinstance(element, overpy.Node):
        lat, lon = element.lat, element.lon
        geom_type = 'node'
    elif isinstance(element, overpy.Way):
        lat, lon = element.center_lat, element.center_lon
        geom_type = 'way'
    else:  # Relation
        lat, lon = element.center_lat, element.center_lon
        geom_type = 'relation'
    
    data.append({
        'osm_id': element.id,
        'name': name,
        'amenity_type': amenity_type,
        'social_facility_type': social_facility_type,
        'geom_type': geom_type,
        'geometry': Point(lon, lat)
    })

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")

# Display the first few rows of the GeoDataFrame
print(gdf.head())
print(f'\nNumber of POIs: {len(gdf)}')

# Write the GeoDataFrame to a shapefile
pyogrio.write_dataframe(gdf, f'{Path.cwd()}\data\\pois_hd_social_facility_osm.shp')

       osm_id                                  name     amenity_type  \
0  2793896116   SRH Pflege Heidelberg Junges Wohnen     nursing_home   
1  5227329870  AWO-Seniorenzentrum Im Kranichgarten  social_facility   
2  7551958037             Seniorenzentrum Kirchheim  social_facility   
3  7553093487      Südstadtresidenz Caroline Sammet  social_facility   
4  9964284763             Altes reformiertes Spital  social_facility   

  social_facility_type geom_type                  geometry  
0                  N/A      node  POINT (8.65301 49.41410)  
1      assisted_living      node  POINT (8.64399 49.39486)  
2         nursing_home      node  POINT (8.66947 49.37891)  
3         nursing_home      node  POINT (8.68689 49.38984)  
4      assisted_living      node  POINT (8.69783 49.40896)  

Number of POIs: 15


Access Kinder Gardens in Heidelberg via overpass API

In [3]:
# Build Query for Kindergartens in Heidelberg
result = api.query("""
    [out:json];
    area["name"="Heidelberg"]["admin_level"="6"]->.searchArea;
    (
      node["amenity"="kindergarten"](area.searchArea);
      way["amenity"="kindergarten"](area.searchArea);
      relation["amenity"="kindergarten"](area.searchArea);
    );
    out center;
""")

# Prepare data for GeoDataFrame
data = []

for element in result.nodes + result.ways + result.relations:
    name = element.tags.get('name', 'N/A')
    amenity_type = element.tags.get('amenity', 'N/A')
    
    if isinstance(element, overpy.Node):
        lat, lon = element.lat, element.lon
        osm_id = f"node/{element.id}"
        element_type = 'node'
    elif isinstance(element, overpy.Way):
        lat, lon = element.center_lat, element.center_lon
        osm_id = f"way/{element.id}"
        element_type = 'way'
    else:  # Relation
        lat, lon = element.center_lat, element.center_lon
        osm_id = f"relation/{element.id}"
        element_type = 'relation'
    
    data.append({
        'osm_id': osm_id,
        'name': name,
        'amenity_type': amenity_type,
        'geom_type': element_type,
        'geometry': Point(lon, lat)
    })

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(data, crs="EPSG:4326", geometry='geometry')
gdf['priority'] = gdf['geom_type'].map({'node': 0, 'way': 1, 'relation': 2})
gdf = gdf.sort_values(['name', 'priority'])

# Function to filter nearby duplicates using GeoPandas
def filter_nearby_duplicates(group):
    if len(group) == 1:
        return group
    
    # Convert to a projected CRS for accurate distance calculation
    group_projected = group.to_crs(epsg=25832)
    
    # Create a 100m buffer around the first point
    buffer = group_projected.iloc[0].geometry.buffer(100)
    
    # Select points that are outside this buffer
    outside_buffer = group_projected[~group_projected.geometry.within(buffer)]
    
    # Combine the first point with those outside the buffer
    result = pd.concat([group_projected.iloc[[0]], outside_buffer])
    
    # Convert back to original CRS
    return result.to_crs(gdf.crs)

# Apply the filter
gdf = gdf.groupby('name', group_keys=False).apply(filter_nearby_duplicates)

# Drop the temporary 'priority' column
gdf = gdf.drop(columns=['priority'])

# Display the first few rows of the GeoDataFrame
print(gdf.head())
print(f'\nNumber of POIs: {len(gdf)}')

# Write the GeoDataFrame to a shapefile
pyogrio.write_dataframe(gdf, f'{Path.cwd()}\data\\pois_hd_kinder_osm.shp')

              osm_id                                               name  \
15   node/1029589033                     AWO Kindertagesstätte Bergheim   
41   node/5361822166                             AWO-Kita Badischer Hof   
61   node/9503661064                                     Am Paradeplatz   
20   node/1918196616                                              Arche   
103    way/252021712  Champini Sport- & Bewegungskita Heidelberg Sch...   

     amenity_type geom_type                  geometry  
15   kindergarten      node  POINT (8.67796 49.40692)  
41   kindergarten      node  POINT (8.66630 49.37787)  
61   kindergarten      node  POINT (8.68317 49.38769)  
20   kindergarten      node  POINT (8.66857 49.38471)  
103  kindergarten       way  POINT (8.77192 49.41276)  

Number of POIs: 148


Access Trees from OpenStreetMap via overpass API

In [4]:
# Build Query for Trees in Heidelberg
result = api.query("""
    [out:json];
    area["name"="Heidelberg"]["admin_level"="6"]->.searchArea;
    (
      node["natural"="tree"](area.searchArea);
      way["natural"="tree"](area.searchArea);
      relation["natural"="tree"](area.searchArea);
    );
    out center;
""")

# Prepare data for GeoDataFrame
data = []

for element in result.nodes + result.ways + result.relations:
    name = element.tags.get('name', 'N/A')
    
    if isinstance(element, overpy.Node):
        lat, lon = element.lat, element.lon
        geom_type = 'node'
    elif isinstance(element, overpy.Way):
        lat, lon = element.center_lat, element.center_lon
        geom_type = 'way'
    else:  # Relation
        lat, lon = element.center_lat, element.center_lon
        geom_type = 'relation'
    
    data.append({
        'osm_id': element.id,
        'name': name,
        'geom_type': geom_type,
        'geometry': Point(lon, lat),
        'tags' : element.tags
        
    })

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")

# Display the first few rows of the GeoDataFrame
print(gdf.head())
print(f'\nNumber of POIs: {len(gdf)}')

# Export the GeoDataFrame to a shapefile
pyogrio.write_dataframe(gdf, f'{Path.cwd()}\data\\pois_hd_trees_osm.shp')

      osm_id          name geom_type                  geometry  \
0  568721264  Traubeneiche      node  POINT (8.71082 49.43980)   
1  603021904           N/A      node  POINT (8.67120 49.41617)   
2  603021907           N/A      node  POINT (8.67132 49.41617)   
3  603021909           N/A      node  POINT (8.67131 49.41625)   
4  603021910           N/A      node  POINT (8.67120 49.41625)   

                                                tags  
0  {'board_type': 'plants', 'information': 'board...  
1  {'circumference': '0.5', 'denotation': 'urban'...  
2  {'circumference': '0.5', 'denotation': 'urban'...  
3  {'circumference': '0.5', 'denotation': 'urban'...  
4  {'circumference': '0.5', 'denotation': 'urban'...  

Number of POIs: 6522
